In [10]:
import os
import asyncio
from typing import Optional
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough

load_dotenv()

llm = ChatOpenAI(
    model="qwen3-max-2026-01-23",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0,
)

## --- 定义独立链 ---
## 这三个链代表可以并执行的不同任务。
summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [("system", "简洁地总结以下主题："), ("user", "{topic}")]
    )
    | llm
    | StrOutputParser()
)

questions_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [("system", "生成关于以下主题的三个有趣问题："), ("user", "{topic}")]
    )
    | llm
    | StrOutputParser()
)

terms_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "从以下主题中识别 5-10 个关键术语，用逗号分隔："),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

## --- 构建并行 + 综合链 ---
## 1. 定义要并行运行的任务块。这些的结果，
## 以及原始主题，将被馈送到下一步。
map_chain = RunnableParallel(
    {
        "summary": summarize_chain,
        "questions": questions_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough(),  # 传递原始主题
    }
)

## 2. 定义将组合并行结果的最终综合提示词。
synthesis_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """基于以下信息：
摘要：{summary}
相关问题：{questions}
关键术语：{key_terms}
综合一个全面的答案。""",
        ),
        ("user", "原始主题：{topic}"),
    ]
)

## 3. 通过将并行结果直接管道化
## 到综合提示词，然后是 LLM 和输出解析器，构建完整链。
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()


## --- 运行链 ---
async def run_parallel_example(topic: str) -> None:
    """
    异步调用具有特定主题的并行处理链
    并行打印综合结果。
    参数：
        topic: 要由 LangChain 链处理的输入主题。
    """
    if not llm:
        print("LLM 未初始化。无法运行示例。")
        return

    print(f"\n--- 运行主题的并行 LangChain 示例：'{topic}' ---")
    try:
        # `ainvoke` 的输入是单个 'topic' 字符串，
        # 然后传递给 `map_chain` 中的每个可运行对象。
        response = await full_parallel_chain.ainvoke(topic)
        print("\n--- 最终响应 ---")
        print(response)
    except Exception as e:
        print(f"\n链执行期间发生错误：{e}")


if __name__ == "__main__":
    test_topic = "太空探索的历史"
    # 在 Jupyter Notebook 中，事件循环已经在运行 — 使用顶层 await
    await run_parallel_example(test_topic)


--- 运行主题的并行 LangChain 示例：'太空探索的历史' ---

--- 最终响应 ---
太空探索的历史是一部由技术突破、地缘政治竞争、国际合作与商业创新交织而成的壮丽篇章。其发展大致可分为三个阶段：冷战驱动的“太空竞赛”时期、多国协作的和平利用阶段，以及21世纪以来日益蓬勃的商业化探索时代。

**第一阶段（1950s–1970s）：冷战催生的太空竞赛**  
太空探索的现代起点通常定于1957年——苏联成功发射人类历史上第一颗人造卫星“斯普特尼克1号”，不仅震惊世界，也标志着“太空时代”的开启。这一成就直接刺激了美国加速其航天计划，并促成了NASA的成立（1958年）。随后，1961年苏联宇航员尤里·加加林乘坐“东方一号”进入轨道，成为首位进入太空的人类；作为回应，美国启动雄心勃勃的阿波罗计划，最终在1969年通过“阿波罗11号”任务实现尼尔·阿姆斯特朗和巴兹·奥尔德林的人类首次月球登陆。这一时期的火箭技术（如苏联的R-7和美国的土星五号）飞速发展，载人航天从理论走向现实。

值得注意的是，许多被忽视的先驱者也为早期探索奠定了基础。例如，美国数学家凯瑟琳·约翰逊精确计算了水星和阿波罗任务的轨道；苏联工程师谢尔盖·科罗廖夫是“斯普特尼克”和加加林飞行背后的总设计师，却因政治原因长期隐姓埋名；还有女性宇航员瓦莲京娜·捷列什科娃（1963年首位进入太空的女性）等，都在关键时刻推动了人类迈向太空的步伐。

**第二阶段（1970s–1990s）：从对抗走向合作**  
随着冷战缓和，美苏开始探索合作可能。1975年“阿波罗-联盟测试计划”实现了两国飞船在轨对接，象征性地结束了太空竞赛。此后，双方各自发展空间站：苏联的“礼炮号”和“和平号”，美国的“天空实验室”。1980年代，美国推出可重复使用的航天飞机，提升了近地轨道任务的灵活性。到1990年代，国际合作成为主流——以美国、俄罗斯、欧洲、日本和加拿大为主导的国际空间站（ISS）于1998年开始组装，成为人类在轨长期驻留和科学研究的重要平台。

**第三阶段（2000s至今）：深空探测与商业航天崛起**  
进入21世纪，无人深空探测取得重大突破：旅行者1号和2号飞出日球层，火星车（如“好奇号”“毅力号”）在红色星球上寻找生命痕迹，新视野号飞掠冥王星，詹姆斯·韦伯太空望远镜开启宇宙观测新纪元。与此同时，